<a href="https://colab.research.google.com/github/xKDR/Shedding-light-on-the-Russia-Ukraine-war/blob/main/reproducible_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WORK IN PROGRESS

In [20]:
using Pkg
Pkg.add(["GeoJSON", "NighttimeLights", "Rasters", "CSV", "Dates", "DataFrames"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
  17718.1 ms  ? NighttimeLights
  18654.4 ms  ? Rasters → RastersMakieExt


In [3]:
;git clone https://github.com/xKDR/Shedding-light-on-the-Russia-Ukraine-war.git

Cloning into 'Shedding-light-on-the-Russia-Ukraine-war'...
Updating files: 100% (272/272), done.


In [21]:
using GeoJSON
using GeoDataFrames
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [22]:
# Define path constants
FIGURES_DIRECTORY = "figures"
TABLES_DIRECTORY = "tables"
VECTOR_DIRECTORY = "data/vector"
TABULAR_DIRECTORY = "data/tabular"
RADIANCE_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"
CFOBS_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"

"/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"

In [24]:
function read_vector(filename)
# Use GeoJSON.read, which expects an IO
    geo = open(filename) do f
        GeoJSON.read(f)
    end

    # Now extract features
    features = geo.features

    # Build DataFrame with properties + geometry
    return DataFrame([merge(Dict(f.properties), Dict(:geometry => f.geometry)) for f in features])
end

read_vector (generic function with 1 method)

In [25]:
russia_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

Row,geometry,NAME_1,NL_NAME_1,GID_0,VARNAME_1,HASC_1,ISO_1,path,TYPE_1,GID_1,COUNTRY,CC_1,ENGTYPE_1,layer
,Abstract…,String,String,String,String,String,String,String,String,String,String,String,String,String
1,2D Polygon,Chukot1,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
2,2D Polygon,Chukot2_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
3,2D Polygon,Chukot3,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
4,2D Polygon,Chukot4,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
5,2D Polygon,Chukot5_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
6,2D Polygon,Krasnoyarsk1,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
7,2D Polygon,Krasnoyarsk2,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
8,2D MultiPolygon,Krasnoyarsk3,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
9,2D Polygon,Krasnoyarsk4,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states


In [ ]:
function safe_centre_of_mass(slice::AbstractArray, state_name::String)
    s = sum(skipmissing(slice))
    if ismissing(s) || isnan(s) || s == 0.0f0
        return [NaN, NaN]
    end
    try
        return NighttimeLights.centre_of_mass(slice)
    catch e
        if isa(e, InexactError)
            println("Warning: CoM calculation failed for '$state_name'. Returning [NaN, NaN].")
            return [NaN, NaN]
        else
            rethrow(e)
        end
    end
end

In [ ]:
function process_state(state_row::DataFrameRow)
    state_name = state_row.NAME_1
    state_geometry = state_row.geometry

    radiance, cfobs = readnl(state_geometry, START_DATE, END_DATE;
                            rad_path=RAD_PATH,
                            cf_path=CF_PATH) #,resample_factor = 2

    radiance = clean_complete(radiance, cfobs; bgthreshold=4)
    cfobs = nothing # Release memory

    radiance = Raster(Float32.(radiance); missingval=missing)
    radiance[radiance .> 100.0f0] .= 0.0f0 # Cap high radiance values
    radiance = mask(radiance; with=state_geometry)

    dates = Date.(dims(radiance, Ti).val)
    num_timesteps = length(dates)

    if num_timesteps == 0
        println("Warning: No valid data for '$state_name' after cleaning.")
        return DataFrame()
    end

    agg_radiance = [sum(skipmissing(view(radiance, Ti(i)))) for i in 1:num_timesteps]
    com = [safe_centre_of_mass(view(radiance, Ti(i)), state_name) for i in 1:num_timesteps]

    return DataFrame(; dates, aggregate_radiance=agg_radiance, com, state_name)
end

In [ ]:
all_states_df = DataFrame()

for i in start_index:nrow(shp)
    state_row = shp[i, :]
    state_name = state_row.NAME_1

    if state_name in IGNORED_STATES
        println("Skipping state ($i/$(nrow(shp))): $state_name")
        continue
    end

    println("--- Processing state $i/$(nrow(shp)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(all_states_df, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(all_states_df)
    println("Aggregating results...")
    grouped = groupby(all_states_df, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")
